## Python 爬虫原理(3):爬虫与反爬虫
　　参考：[可能是最全的反爬虫及应对方案](https://mp.weixin.qq.com/s?__biz=MzAxMjcyNjE5MQ==&mid=2650491087&idx=2&sn=9eb62b1c6e396a13bcc3cbf9ffd175d2&chksm=83a2920fb4d51b19cdf0eca97cd34671f601cd216f4c736097566bf0f59d1ca80c71ead2fdfa&mpshare=1&scene=1&srcid=1114jyvNSpLHJwjcy13x9PIh&sharer_sharetime=1605351715393&sharer_shareid=03afa65f04640bef043a74559a39ca4a&key=516e28ba4c9b5ddf6dda0837f1c4ef91d6b6fe777a80bd96498779e601a9d615def084ba622f7987e96e39e2def6509107fcb50526507f4d838d69faedca440eead9e1ded9eb8335ef40f66ad2a8f46d6f894eb1306ff67da0eaf8339e130c38e63340a9c93fa8c5eebb7b9c54d8531f281cb12c5f61a6bf0f929ffa545440d9&ascene=1&uin=MTI5MjM5ODU0MQ%3D%3D&devicetype=Windows+10+x64&version=6300002f&lang=zh_CN&exportkey=AetzqPDWdPH33EunqAj7ab8%3D&pass_ticket=j3hgdKhVpGaa9MrOegOb9AcVKizqUJBdXKh851HC9HbJR6iVrZG7si9cv4h8ZcPP&wx_header=0)<br>

**1、爬虫与反爬虫介绍**<br>
爬虫：使用任何技术手段，批量获取网站信息的一种方式。<br>
反爬虫：使用任何技术手段，阻止别人批量获取自己网站信息的一种方式。

**2、反爬策略汇总**<br>
1)、通过UA 识别爬虫<br>
有些爬虫的UA是特殊的，与正常浏览器的不一样，可通过识别特征UA，直接封掉爬虫请求<br>
2)、设置IP访问频率，如果超过一定频率，弹出验证码<br>
如果输入正确的验证码，则放行，如果没有输入，则拉入禁止一段时间，如果超过禁爬时间，再次错发发验证码，则拉入黑名单。当然根据具体的业务，为不同场景设置不同阈值，比如登陆用户和非登陆用户，请求是否含有refer。<br>
3)、通过并发识别爬虫<br>
有些爬虫的并发是很高的，统计并发最高的IP，加入黑名单（或者直接封掉爬虫IP所在C段）<br>
4)、请求的时间窗口过滤统计<br>
爬虫爬取网页的频率都是比较固定的，不像人去访问网页，中间的间隔时间比较无规则，所以我们可以给每个IP地址建立一个时间窗口，记录IP地址最近12次访问时间，每记录一次就滑动一次窗口，比较最近访问时间和当前时间，如果间隔时间很长判断不是爬虫，清除时间窗口，如果间隔不长，就回溯计算指定时间段的访问频率，如果访问频率超过阀值，就转向验证码页面让用户填写验证码<br>
5)、限制单个ip/api token的访问量<br>
比如15分钟限制访问页面180次，具体标准可参考一些大型网站的公开api，如twitter api，对于抓取用户公开信息的爬虫要格外敏感<br>
6)、识别出合法爬虫<br>
对http头agent进行验证，是否标记为、百度的spider，严格一点的话应该判别来源IP是否为、baidu的爬虫IP，这些IP在网上都可以找到。校验出来IP不在白名单就可以阻止访问内容。<br>
7)、蜜罐资源<br>
爬虫解析离不开正则匹配，适当在页面添加一些正常浏览器浏览访问不到的资源，一旦有ip访问，过滤下头部是不是搜素引擎的蜘蛛，不是就可以直接封了。比如说隐式链接。<br>
8)、web反爬虫策略：https://www.cnblogs.com/liinux/p/5558943.html#top （web反爬虫策略）<br>
9)、高级反爬：https://www.jianshu.com/p/f79d8e674768 (根据自定义映射关系表加密解密)<br>

**3、爬虫策略汇总**<br>
1)、设置下载延迟，比如数字设置为5秒，越大越安全<br>
2)、禁止Cookie，某些网站会通过Cookie识别用户身份，禁用后使得服务器无法识别爬虫轨迹<br>
3)、使用user agent池。也就是每次发送的时候随机从池中选择不一样的浏览器头信息，防止暴露爬虫身份<br>
4)、使用IP池，这个需要大量的IP资源，可以通过抓取网上免费公开的IP建成自有的IP代理池。<br>
5)、分布式爬取，这个是针对大型爬虫系统的，实现一个分布式的爬虫，主要为以下几个步骤：<br> 
a、基本的http抓取工具，如scrapy； <br>
b、避免重复抓取网页，如Bloom Filter； <br>
c、维护一个所有集群机器能够有效分享的分布式队列； <br>
d、将分布式队列和Scrapy的结合； <br>
e、后续处理，网页析取(如python-goose)，存储(如Mongodb)。<br>
6)、模拟登录—浏览器登录的爬取<br>
设置一个cookie处理对象，它负责将cookie添加到http请求中，并能从http响应中得到cookie，向网站登录页面发送一个请求Request, 包括登录url，POST请求的数据，Http header利用urllib2.urlopen发送请求，接收WEB服务器的Response。